### Using Google Places API Places - Text Search to obtain gyms data in a neighborhood
Documentation of Places API: https://developers.google.com/places/web-service/intro

Python Client for Google Maps Services: https://github.com/googlemaps/google-maps-services-python 

In [3]:
import googlemaps
import pprint
import time
import numpy as np
import pandas as pd
import os
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html
from pandas.io.json import json_normalize

#define API key
API_KEY = ''

#define out client
gmaps = googlemaps.Client(key = API_KEY)

Define all neighboods and associated zip codes in Manhattan, based on this mapping from NYC.
https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm

Manhattan zipcodes

In [42]:
Central_Harlem = ["10026", "10027", "10030", "10037", "10039"]
Chelsea_Clinton = ["10001", '10011', '10018', '10019', '10020', '10036']
East_Harlem = ['10029', '10035']
Gramercy_Park_Murray_Hill = ['10010', '10016', '10017', '10022']
Greenwich_Village_Soho = ["10012", "10013", "10014"]
Lower_Manhattan = ['10004', '10005', '10006', '10007', '10038', '10280']
Lower_East_Side = ['10002', '10003', '10009']
Upper_East_Side = ['10021', '10028', '10044', '10065', '10075', '10128']
Upper_West_Side = ['10023', '10024', '10025']
Inwood_and_Washington_Heights = ['10031', '10032', '10033', '10034', '10040']

Bronx

In [43]:
Bronx_Central_Bronx = ['10453', '10457', '10460']
Bronx_Park_Fordham = ['10458', '10467', '10468']
High_Bridge_and_Morrisania = ['10451', '10452', '10456']
Hunts_Point_and_Mott_Haven = ['10454', '10455', '10459', '10474']
Kingsbridge_and_Riverdale = ['10463', '10471']
Northeast_Bronx = ['10466', '10469', '10470', '10475']
Southeast_Bronx = ['10461', '10462', '10464', '10465', '10472', '10473']

Brooklyn

In [44]:
Brooklyn_Central_Brooklyn = ['11212', '11213', '11216', '11233', '11238']
Southwest_Brooklyn = ['11209', '11214', '11228']
Borough_Park = ['11204', '11218', '11219', '11230']
Canarsie_and_Flatlands = ['11234', '11236', '11239']
Southern_Brooklyn = ['11223', '11224', '11229', '11235']
Northwest_Brooklyn = ['11201', '11205', '11215', '11217', '11231']
Flatbush = ['11203', '11210', '11225', '11226']
East_New_York_and_New_Lots = ['11207', '11208']
Greenpoint = ['11211', '11222']
Sunset_Park = ['11220', '11232']
Bushwick_Williamsburg = ['11206', '11221', '11237']

Queens

In [45]:
Northeast_Queens = ['11361', '11362', '11363', '11364']
North_Queens = ['11354', '11355', '11356', '11357', '11358', '11359', '11360']
Central_Queens = ['11365', '11366', '11367']
Jamaica = ['11412', '11423', '11432', '11433', '11434', '11435', '11436']
Northwest_Queens = ['11101', '11102', '11103', '11104', '11105', '11106']
West_Central_Queens = ['11374', '11375', '11379', '11385']
Rockaways = ['11691', '11692', '11693', '11694', '11695', '11697']
Southeast_Queens = ['11004', '11005', '11411', '11413', '11422', '11426', '11427', '11428', '11429']
Southwest_Queens = ['11414', '11415', '11416', '11417', '11418', '11419', '11420', '11421']
West_Queens = ['11368', '11369', '11370', '11372', '11373', '11377', '11378']

Staten Island

In [66]:
Port_Richmond = ['10302', '10303', '10310']
South_Shore = ['10306', '10307', '10308', '10309', '10312']
Stapleton_and_St_George = ['10301', '10304', '10305']
Mid_Island = ['10314']

All nyc zip codes


In [74]:
nyc_zipcodes = {'Brooklyn': Brooklyn_Central_Brooklyn + Southwest_Brooklyn + Borough_Park + \
                              Canarsie_and_Flatlands + Southern_Brooklyn + Northwest_Brooklyn + \
                              Flatbush + East_New_York_and_New_Lots + Greenpoint + Sunset_Park + \
                              Bushwick_Williamsburg,
                  'Bronx': Bronx_Central_Bronx + Bronx_Park_Fordham + High_Bridge_and_Morrisania + \
                           Hunts_Point_and_Mott_Haven + Kingsbridge_and_Riverdale + Northeast_Bronx + \
                           Southeast_Bronx,
                  'Manhattan': Central_Harlem + Chelsea_Clinton + East_Harlem + Gramercy_Park_Murray_Hill + \
                                Greenwich_Village_Soho + Lower_Manhattan + Lower_East_Side + Upper_East_Side + \
                                Upper_West_Side + Inwood_and_Washington_Heights,
                  'Queens': Northeast_Queens + North_Queens + Central_Queens + Jamaica + Northwest_Queens + \
                            West_Central_Queens + Rockaways + Southeast_Queens + Southwest_Queens + West_Queens,
                  'Staten_Island': Port_Richmond + South_Shore + Stapleton_and_St_George + Mid_Island
                     }

In [76]:
nyc_zipcodes.keys()

dict_keys(['Brooklyn', 'Bronx', 'Manhattan', 'Queens', 'Staten_Island'])

Create functions to make three page requests and return results, which gives a maximum of 60 results. 

In [4]:
def first_page_request(query_text, establishment_type):
    
    #make initial request using the text search function 
    places_result = gmaps.places(query=query_text, type=establishment_type)
    
    #if the request status is OK, print number of results
    if places_result['status'] == "OK":
        #create query and page keys to keep track of our requests
        places_result['query'] = query_text
        places_result['page'] = 1
        print("\nPage 1 of {}:".format(query_text))
        print("status: {}".format(places_result['status']))
        print("number of results: {}".format(len(places_result['results'])))
        
        #create a next page token key even if null to keep all keys of a request. 
        places_result['next_page_token'] = places_result.get('next_page_token')
        print("next_page_token: {}".format(places_result['next_page_token']))
        
        return places_result.copy()
        
    else: 
        print("Bad Request")
        return None

def second_page_request(query_text, establishment_type, page_1):
    
    #get the next 20 results
    next_page_token = page_1.get('next_page_token')
    if next_page_token:
        time.sleep(2)
        places_result2 = gmaps.places(query=query_text, type=establishment_type, page_token = next_page_token)
        if places_result2['status'] == "OK":
            
            #create query and page keys to keep track of our second request
            places_result2['query'] = query_text
            places_result2['page'] = 2
            print("\nPage 2 of {}:".format(query_text))
            print("status: {}".format(places_result2['status']))
            print("number of results: {}".format(len(places_result2['results'])))
    
            #create a next page token key if null to keep all keys of a request. 
            places_result2['next_page_token'] = places_result2.get('next_page_token')
            print("next_page_token: {}".format(places_result2['next_page_token']))

            return places_result2.copy()
    else:
        print("\nNext page result is {}:".format(next_page_token))
        return None

def third_page_request(query_text, establishment_type, page_2):
    
    #get the next 20 results
    next_page_token = page_2.get('next_page_token')
    if next_page_token:
        time.sleep(2)
        places_result3 = gmaps.places(query=query_text, type=establishment_type, page_token = next_page_token)
        
        if places_result3['status'] == "OK":
            #create query and page keys to keep track of our third request and append page to pages list
            places_result3['query'] = query_text
            places_result3['page'] = 3
            print("\nPage 3 of {}:".format(query_text))
            print("status: {}".format(places_result3['status']))
            print("number of results: {}".format(len(places_result3['results'])))
            
            #create a next page token key if null to keep all keys of a request. 
            places_result3['next_page_token'] = places_result3.get('next_page_token')
            print("next_page_token: {}".format(places_result3['next_page_token']))
            
            return places_result3.copy()
    else:
        print("\nNext page result is {}:".format(next_page_token))
        return None

#### Make requests for each zip code in the neighborhood and save return results.  
We will also normalize the JSON results as pandas dataframes, which would flatten all keys into columns and values as rows, for a tabular output later.

In [285]:
#list to hold page results from requests
pages = []

#define parameters for our request: neighborhood zipcodes and type of establishment
zip_codes = Mid_Island
establishment_type = 'gym'

for zipcode in zip_codes:
    
    #pause script for 2 seconds for next zip code query
    time.sleep(2)
    
    #request first page results
    query_text = "gyms in " + zipcode
    place_result = dict()
    place_result = first_page_request(query_text=query_text, establishment_type=establishment_type)
    
    
    
    #if there are returned results, append to pages list. 
    if place_result:
        pages.append(json_normalize(place_result['results']))


        #request second page results
        place_result2 = dict()
        place_result2 = second_page_request(query_text=query_text, establishment_type=establishment_type, 
                                           page_1=place_result)

        #if there are returned results, append to pages list. 
        if (place_result2 and place_result2['results']):
            pages.append(json_normalize(place_result2['results']))
    

            #request third and last page of results if available:
            place_result3 = dict()
            place_result3 = third_page_request(query_text=query_text, establishment_type=establishment_type, 
                                               page_2=place_result2)

            #if there are returned results, append to pages list
            if (place_result3 and place_result3['results']):
                pages.append(json_normalize(place_result3['results']))
                
    else: continue
                



Page 1 of gyms in 10314:
status: OK
number of results: 20
next_page_token: CpQCAQEAAD37YNya7Jih0MsosEIqdOaNQCK_CVzkaSmtf2HidfUpsV_FEHY-KTNupdGTtgn7f0nz0g1A0PPWfxpnz5_5wBBSb8B1nzzducqdY7Fvq6y83-nP4XQPkv4JBbehPzLa_-xnq-wxBnEfQnQkwSw6ER8wYrutk2jsmRn9o8ZlBrNjCC1h_Db5CeWkA5srmoG6z74erXz90Sw3nRxEgAmjJRCVgBuiqC8oxeucHKpHQw-PY3yGCgmdtT3MZWRYSAPa-Kbxz24S5GVZK2zsFz7LGH5ahGOFwmVJ5P0F3dI-EGcqjYv9W7CLTJ45J5M1coOtciSz5CTQSU1dohVGL0dWUtIaHllWfLbKuVD2cbJwizH2EhCAMpwlv27cOdn8MAsNNSksGhRA-EtTfYU2I0VTdb4OIPgT4NhwKg

Page 2 of gyms in 10314:
status: OK
number of results: 20
next_page_token: CrQDoQEAAAARa2lTEnQKSiM0MwSHKhQSWuKfB36YzK15lQP__bpRoSrb3zUY27kWtj_CLlB-pjmYxK7CMGu9u4hGEa2KfnTYbhuXca-9I-65OSavQHGMgOVZ0RjVgTml6Vi8s4Qad_6i6HSKAm5TRKfDwfKTKVMHWoDkjASU3moz--xeIrQywFO8PbfPFC6OzC6tt88Tr8LY3XKQleOnR29kfvRr1qR0U4Qq3TDXdRXqltw55JzwlHUe40GlIRkJpLo_0GWbmHPAMFGJKCLrXoGf8Rvmvo45rQuQaKZGvczRGCle1JB1lKBKUApG0L0aD8y-Gd4977ZNCipZLfkmOsT8L1VqcHhwmQJ2kbdFNKQZ1-NDSiTBXCWfJi7-uDMhZC2TtK1pdGGyMOcwBOSnPHoFXfFkL6LxDfMhE

#### Look at the dataframe results, which was normalized from the JSON file to a table format with columns and rows. 

In [286]:
#print number files saved as an dataframes
print("number files saved as an dataframes: {}\n\nShape of each file:".format(len(pages)))

for page in pages:
    print(page.shape)

number files saved as an dataframes: 3

Shape of each file:
(20, 19)
(20, 19)
(20, 19)


#### Concat the dataframes into a single dataframe

In [287]:
df_files = pd.concat(pages, ignore_index=True, sort=False)
df_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 19 columns):
formatted_address                  60 non-null object
geometry.location.lat              60 non-null float64
geometry.location.lng              60 non-null float64
geometry.viewport.northeast.lat    60 non-null float64
geometry.viewport.northeast.lng    60 non-null float64
geometry.viewport.southwest.lat    60 non-null float64
geometry.viewport.southwest.lng    60 non-null float64
icon                               60 non-null object
id                                 60 non-null object
name                               60 non-null object
opening_hours.open_now             45 non-null object
photos                             50 non-null object
place_id                           60 non-null object
plus_code.compound_code            60 non-null object
plus_code.global_code              60 non-null object
rating                             60 non-null float64
reference                 

#### Identify the number of duplicates in the neighborhood search

In [288]:
#count number of duplicated records
print("\nNumber of duplicates: {}". format(df_files['place_id'].duplicated().sum()))


Number of duplicates: 0


#### Drop duplicated records.

In [289]:
df_files.drop_duplicates(subset='place_id', inplace=True)
print("After dropping duplicates, data shape is: {}".format(df_files.shape))
print("Number of duplicates: {}". format(df_files['place_id'].duplicated().sum()))

After dropping duplicates, data shape is: (60, 19)
Number of duplicates: 0


In [290]:
df_files

,formatted_address,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,icon,id,name,opening_hours.open_now,photos,place_id,plus_code.compound_code,plus_code.global_code,rating,reference,types,user_ratings_total
0,"2385 Richmond Ave, Staten Island, NY 10314, Un...",40.589564,-74.165654,40.590851,-74.164363,40.588152,-74.167063,https://maps.gstatic.com/mapfiles/place_api/ic...,c4dd4a24baceafb558f62bd5dbacaaaaaaa23cb4,Retro Fitness,True,"[{'height': 608, 'html_attributions': ['<a hre...",ChIJkTbIdnxMwokRrDXwZI_ICWw,"HRQM+RP Staten Island, New York",87G7HRQM+RP,3.0,ChIJkTbIdnxMwokRrDXwZI_ICWw,"[gym, health, point_of_interest, establishment]",115
1,"2409 Richmond Ave, Staten Island, NY 10314, Un...",40.588842,-74.166835,40.590234,-74.165561,40.587535,-74.168261,https://maps.gstatic.com/mapfiles/place_api/ic...,a82d2ce7e6f4215809c510e24a08967947843dae,Orangetheory Fitness,True,"[{'height': 1153, 'html_attributions': ['<a hr...",ChIJi1cDPXxMwokRFn5Vhca79fU,"HRQM+G7 Staten Island, New York",87G7HRQM+G7,4.9,ChIJi1cDPXxMwokRFn5Vhca79fU,"[gym, health, point_of_interest, establishment]",259
2,"300 West Service Road, Staten Island, NY 10314...",40.586098,-74.193740,40.587398,-74.192273,40.584699,-74.194973,https://maps.gstatic.com/mapfiles/place_api/ic...,597f22a112e0fe3a611fef9ec594c4c217989657,New York Sports Clubs,True,"[{'height': 1266, 'html_attributions': ['<a hr...",ChIJ_24MBWBLwokR9HbSuyXNPgw,"HRP4+CG Staten Island, New York",87G7HRP4+CG,3.0,ChIJ_24MBWBLwokR9HbSuyXNPgw,"[gym, health, point_of_interest, establishment]",82
3,"2040 Forest Ave, Staten Island, NY 10303, Unit...",40.624994,-74.156046,40.626314,-74.154744,40.623615,-74.157444,https://maps.gstatic.com/mapfiles/place_api/ic...,d82535cda088e30bd5e0d4c2d5e499c1c24d19f6,Planet Fitness,True,"[{'height': 2176, 'html_attributions': ['<a hr...",ChIJjfJNOLJNwokRS89EzDRCeNk,"JRFV+XH Staten Island, New York",87G7JRFV+XH,4.2,ChIJjfJNOLJNwokRS89EzDRCeNk,"[gym, spa, health, point_of_interest, establis...",340
4,"1775 South Ave, Staten Island, NY 10314, Unite...",40.600113,-74.193326,40.601510,-74.191880,40.598811,-74.194579,https://maps.gstatic.com/mapfiles/place_api/ic...,ff408c70b25c1f24eb5828d17c12643a710e8f24,Planet Fitness,True,"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJG4aHCpZMwokRz0ieYsxs-Zo,"JR24+2M Staten Island, New York",87G7JR24+2M,3.9,ChIJG4aHCpZMwokRz0ieYsxs-Zo,"[gym, spa, health, point_of_interest, establis...",273
5,"1655 Richmond Ave, Staten Island, NY 10314, Un...",40.607769,-74.162054,40.609037,-74.160755,40.606337,-74.163455,https://maps.gstatic.com/mapfiles/place_api/ic...,a6498e197c712e92c890bcc64b8295e7f6ddab38,Nautilus Women Fitness Center,NaN,NaN,ChIJ1-OrIoJLwokR8D10hmnHb6g,JR5Q+45 New York,87G7JR5Q+45,5.0,ChIJ1-OrIoJLwokR8D10hmnHb6g,"[gym, health, point_of_interest, establishment]",1
6,"1403 Richmond Ave, Staten Island, NY 10314, Un...",40.613959,-74.157432,40.615357,-74.156238,40.612657,-74.158938,https://maps.gstatic.com/mapfiles/place_api/ic...,6813f4cef4c766de5f238074c680126188789537,Transformation Fitness Studio of Staten Island,True,"[{'height': 771, 'html_attributions': ['<a hre...",ChIJtTkKb3lMwokRbm0nRgE1m_Y,"JR7V+H2 Staten Island, New York",87G7JR7V+H2,5.0,ChIJtTkKb3lMwokRbm0nRgE1m_Y,"[gym, health, point_of_interest, establishment]",85
7,"145 E Service Rd, Staten Island, NY 10314, Uni...",40.587940,-74.191059,40.589204,-74.189528,40.586504,-74.192227,https://maps.gstatic.com/mapfiles/place_api/ic...,258981607062f28fa3f1c0d5f846aaa6762aa3fb,LA Fitness,True,"[{'height': 2048, 'html_attributions': ['<a hr...",ChIJB5-0tJ9MwokRs0KREGl4eik,"HRQ5+5H Staten Island, New York",87G7HRQ5+5H,4.1,ChIJB5-0tJ9MwokRs0KREGl4eik,"[gym, spa, health, point_of_interest, establis...",268
8,"268 Watchogue Rd, Staten Island, NY 10314, Uni...",40.616018,-74.138342,40.617411,-74.136982,40.614711,-74.139681,https://maps.gstatic.com/mapfiles/place_a

#### Export result as csv file

In [291]:
#export to excel and csv
df_files.to_csv("gyms/staten_island/zipcodes_{}.csv".format('_'.join(zip_codes)), index=False)
print("\nExported data as csv file: zipcodes_{}.csv".format('_'.join(zip_codes)))


Exported data as csv file: zipcodes_10314.csv


### Combining all of the borough's neighborhood results and removing duplicates

In [292]:
#create a list of a csv file names in directory.
path_to_csv = 'gyms/staten_island//'
csv_files = [pos_csv for pos_csv in os.listdir(path_to_csv) if pos_csv.endswith('.csv')]
csv_files

['zipcodes_10301_10304_10305.csv',
 'zipcodes_10314.csv',
 'zipcodes_10306_10307_10308_10309_10312.csv',
 'zipcodes_10302_10303_10310.csv']

In [293]:
dfs = []
for fn in csv_files:
    dfs.append(pd.read_csv('gyms/staten_island/{}'.format(fn)))
    
df_combined = pd.concat(dfs,ignore_index=True, sort=False)

df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 19 columns):
formatted_address                  312 non-null object
geometry.location.lat              312 non-null float64
geometry.location.lng              312 non-null float64
geometry.viewport.northeast.lat    312 non-null float64
geometry.viewport.northeast.lng    312 non-null float64
geometry.viewport.southwest.lat    312 non-null float64
geometry.viewport.southwest.lng    312 non-null float64
icon                               312 non-null object
id                                 312 non-null object
name                               312 non-null object
opening_hours.open_now             235 non-null object
photos                             248 non-null object
place_id                           312 non-null object
plus_code.compound_code            312 non-null object
plus_code.global_code              312 non-null object
rating                             312 non-null float64
referenc

In [262]:
#remove duplicates
print("\nNumber of duplicates: {}". format(df_combined['place_id'].duplicated().sum()))
df_combined.drop_duplicates(subset='place_id', inplace=True)
print("After dropping duplicates, data shape is: {}".format(df_combined.shape))
print("Number of duplicates: {}". format(df_combined['place_id'].duplicated().sum()))


Number of duplicates: 720
After dropping duplicates, data shape is: (654, 20)
Number of duplicates: 0


In [294]:
#export to excel and csv
df_combined.to_csv("gyms/staten_island//staten_island_gyms_combined.csv", index=False)
print("\nExported data as csv file: staten_island_gyms_combined.csv")


Exported data as csv file: staten_island_gyms_combined.csv


### Combining all boroughs data 

In [118]:
df_all = pd.concat([pd.read_csv('gyms/bronx/bronx_gyms_combined.csv'),
           pd.read_csv('gyms/brooklyn/brooklyn_gyms_combined.csv'),
           pd.read_csv('gyms/manhattan/manhattan_gyms_combined.csv'),
           pd.read_csv('gyms/queens/queens_gyms_combined.csv'),
           pd.read_csv('gyms/staten_island/staten_island_gyms_combined.csv')], 
           sort='False', ignore_index=True)
df_all

,formatted_address,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,icon,id,name,opening_hours.open_now,photos,place_id,plus_code.compound_code,plus_code.global_code,price_level,rating,reference,types,user_ratings_total
0,"298 W 231st St, The Bronx, NY 10463, United St...",40.880539,-73.908334,40.881771,-73.907093,40.879072,-73.909793,https://maps.gstatic.com/mapfiles/place_api/ic...,a3485e7994aa8626891015b990080e6dc9d7b6b3,24 Hour Fitness,True,"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJEfg3t5XzwokRtR2xHxFYi0Q,"V3JR+6M The Bronx, New York",87G8V3JR+6M,NaN,3.4,ChIJEfg3t5XzwokRtR2xHxFYi0Q,"['gym', 'health', 'point_of_interest', 'establ...",187.0
1,"5520 Broadway, The Bronx, NY 10463, United States",40.877351,-73.905654,40.878701,-73.904305,40.876001,-73.907004,https://maps.gstatic.com/mapfiles/place_api/ic...,81389fd5d6b5dd805551b9beb642637143f06a5c,Blink Fitness,True,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJKftkWJHzwokRWTI8LcCn9LI,"V3GV+WP The Bronx, New York",87G8V3GV+WP,NaN,3.8,ChIJKftkWJHzwokRWTI8LcCn9LI,"['gym', 'health', 'point_of_interest', 'establ...",211.0
2,"5500 Broadway, The Bronx, NY 10463, United States",40.876568,-73.906144,40.877848,-73.904852,40.875149,-73.907552,https://maps.gstatic.com/mapfiles/place_api/ic...,93a5f9fa33efd28fa5726eedeaae0a52803eb091,Astral Fitness,True,"[{'height': 864, 'html_attributions': ['<a hre...",ChIJqczPWpHzwokR3qB2P5FDkIU,"V3GV+JG The Bronx, New York",87G8V3GV+JG,NaN,4.2,ChIJqczPWpHzwokR3qB2P5FDkIU,"['gym', 'health', 'point_of_interest', 'establ...",31.0
3,"82 W 225th St, The Bronx, NY 10463, United States",40.874076,-73.909262,40.875480,-73.907841,40.872780,-73.910541,https://maps.gstatic.com/mapfiles/place_api/ic...,912d8b13c868012e545577eeaa8be0762506adfb,Planet Fitness,True,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJr3JBXpLzwokR_dIaA6b0vd8,"V3FR+J7 The Bronx, New York",87G8V3FR+J7,NaN,4.2,ChIJr3JBXpLzwokR_dIaA6b0vd8,"['gym', 'spa', 'health', 'point_of_interest', ...",309.0
4,"453 W 238th St, The Bronx, NY 10463, United St...",40.887410,-73.905057,40.888704,-73.903729,40.886005,-73.906429,https://maps.gstatic.com/mapfiles/place_api/ic...,a272c6a36fa158e12f45a31e7a2bc01896e26c8c,CrossFit La Caja,True,"[{'height': 1334, 'html_attributions': ['<a hr...",ChIJIUCs9LvzwokRLW1jjLy1ILQ,"V3PV+XX The Bronx, New York",87G8V3PV+XX,NaN,4.9,ChIJIUCs9LvzwokRLW1jjLy1ILQ,"['gym', 'health', 'point_of_interest', 'establ...",14.0
5,"451 W 238th St, The Bronx, NY 10463, United St...",40.887441,-73.905033,40.888719,-73.903712,40.886019,-73.906412,https://maps.gstatic.com/mapfiles/place_api/ic...,bd460b8485fe2617f94ef94eb41db87489a7ae9f,Bronx Barbell Club,True,"[{'height': 1067, 'html_attributions': ['<a hr...",ChIJCxuojLvzwokRuDZAKdRM9kM,"V3PV+XX The Bronx, New York",87G8V3PV+XX,NaN,4.6,ChIJCxuojLvzwokRuDZAKdRM9kM,"['gym', 'health', 'point_of_interest', 'establ...",9.0
6,"3333 Henry Hudson Parkway East # 2, The Bronx,...",40.885582,-73.914337,40.886918,-73.912778,40.884218,-73.915478,https://maps.gstatic.com/mapfiles/place_api/ic...,1ece03b83790fbe1f8456195d9c8b6b9ac32d7d4,Whitehall Club,True,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ5eQBQ8DzwokR3TulqszkeVs,V3PP+67 New York,87G8V3PP+67,NaN,4.6,ChIJ5eQBQ8DzwokR3TulqszkeVs,"['gym', 'health', 'point_of_interest', 'establ...",19.0
7,"2600 Netherland Ave, The Bronx, NY 10463, Unit...",40.878628,-73.914568,40.880124,-73.913392,40.877424,-73.916092,https://maps.gstatic.com/mapfiles/place_api/ic...,8a85344ff5f2f8d95a5b7c97fe83bbe2991e5ad7,TCR The Club of Riverdale,True,"[{'height': 1627, 'html_attributions': ['<a hr...",ChIJD-_1fOvzwokRXbzmmmtKaLo,"V3HP+F5 The Bronx, New York",87G8V3HP+F5,NaN,4.0,ChIJD-_1fOvzwokRXbzmmmtKaLo,"['spa', 'restaurant', 'food', 'school', 'gym',...",32.0
8,"5625-2 Broadway, The Bronx, NY 10463, United S...",40.879859,-73.904360,40.881174,-73.902925,40.878474,-

Remove duplicates

In [119]:
#remove duplicates
print("\nNumber of duplicates: {}". format(df_all['place_id'].duplicated().sum()))
df_all.drop_duplicates(subset='place_id', inplace=True)
print("After dropping duplicates, data shape is: {}".format(df_all.shape))
print("Number of duplicates: {}". format(df_all['place_id'].duplicated().sum()))


Number of duplicates: 600
After dropping duplicates, data shape is: (2068, 20)
Number of duplicates: 0


In [120]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2068 entries, 0 to 2663
Data columns (total 20 columns):
formatted_address                  2068 non-null object
geometry.location.lat              2068 non-null float64
geometry.location.lng              2068 non-null float64
geometry.viewport.northeast.lat    2068 non-null float64
geometry.viewport.northeast.lng    2068 non-null float64
geometry.viewport.southwest.lat    2068 non-null float64
geometry.viewport.southwest.lng    2068 non-null float64
icon                               2068 non-null object
id                                 2068 non-null object
name                               2068 non-null object
opening_hours.open_now             1644 non-null object
photos                             1653 non-null object
place_id                           2068 non-null object
plus_code.compound_code            2061 non-null object
plus_code.global_code              2061 non-null object
price_level                        32 non-null 

Parse and clean `formatted_address` column

In [121]:
df_all['formatted_address']

0       298 W 231st St, The Bronx, NY 10463, United St...
1       5520 Broadway, The Bronx, NY 10463, United States
2       5500 Broadway, The Bronx, NY 10463, United States
3       82 W 225th St, The Bronx, NY 10463, United States
4       453 W 238th St, The Bronx, NY 10463, United St...
5       451 W 238th St, The Bronx, NY 10463, United St...
6       3333 Henry Hudson Parkway East # 2, The Bronx,...
7       2600 Netherland Ave, The Bronx, NY 10463, Unit...
8       5625-2 Broadway, The Bronx, NY 10463, United S...
9       3501 Riverdale Ave, The Bronx, NY 10463, Unite...
10      3732 Riverdale Ave, The Bronx, NY 10463, Unite...
11      5500 Broadway, The Bronx, NY 10463, United States
12      3264 Johnson Ave, The Bronx, NY 10463, United ...
13      3265 Johnson Ave, The Bronx, NY 10463, United ...
14      3350 Bailey Ave, The Bronx, NY 10463, United S...
15      3260 Henry Hudson Pkwy #5m, The Bronx, NY 1046...
16      2449 Morris Ave, The Bronx, NY 10468, United S...
17      3170 W

Extract state and zipcode

In [122]:
df_all['formatted_address'].str.extract(pat='([A-Z]{2}) (\d{5})')

,0,1
0,NY,10463
1,NY,10463
2,NY,10463
3,NY,10463
4,NY,10463
5,NY,10463
6,NY,10463
7,NY,10463
8,NY,10463
9,NY,10463


In [123]:
df_all[['state', 'zipcode']] = df_all['formatted_address'].str.extract(pat='([A-Z]{2}) (\d{5})')

Check for values

In [124]:
df_all['state'].value_counts(dropna=False)

NY     1985
NJ       66
CT        8
PA        7
KS        1
NaN       1
Name: state, dtype: int64

In [125]:
df_all['zipcode'].value_counts(dropna=False)

10022    46
10003    43
10023    42
10010    40
11201    37
11215    37
10011    36
10013    35
10028    32
10018    30
10001    30
10025    29
10012    28
11101    28
11385    28
11211    27
10019    27
10016    27
10314    27
11217    27
11238    25
11375    23
10024    22
11209    21
11222    21
10309    20
11221    20
10014    20
11235    20
10128    19
         ..
11561     1
10020     1
10039     1
10155     1
10166     1
10162     1
10552     1
06037     1
11510     1
11096     1
07208     1
10111     1
06418     1
19085     1
11423     1
08007     1
08879     1
10069     1
08034     1
07311     1
11518     1
07093     1
11109     1
11024     1
11560     1
08816     1
11548     1
07670     1
10710     1
NaN       1
Name: zipcode, Length: 258, dtype: int64

Drop records where the state is not NY; there are 83

In [126]:
df_all[df_all['state'] != 'NY']

,formatted_address,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,icon,id,name,...,place_id,plus_code.compound_code,plus_code.global_code,price_level,rating,reference,types,user_ratings_total,state,zipcode
23,"661 E Palisade Ave, Englewood Cliffs, NJ 07632...",40.881093,-73.951753,40.882547,-73.950334,40.879847,-73.953034,https://maps.gstatic.com/mapfiles/place_api/ic...,d641143acde8d0398174a68e5b52a06e0dc1be6e,"Body Elite Fitness, Inc.",...,ChIJMy4z00XxwokRuOhEa_FQstA,"V2JX+C7 Englewood Cliffs, New Jersey",87G8V2JX+C7,NaN,4.8,ChIJMy4z00XxwokRuOhEa_FQstA,"['gym', 'health', 'point_of_interest', 'establ...",22.0,NJ,07632
28,"34-36 S Dean St, Englewood, NJ 07631, United S...",40.892636,-73.975265,40.893950,-73.973813,40.891251,-73.976512,https://maps.gstatic.com/mapfiles/place_api/ic...,faf50b39a7f45039001674e9e6677166f3a8a5fd,New York Sports Clubs,...,ChIJZfKumxjxwokRWfEMnrpRQig,"V2VF+3V Englewood, New Jersey",87G8V2VF+3V,NaN,3.2,ChIJZfKumxjxwokRWfEMnrpRQig,"['gym', 'health', 'point_of_interest', 'establ...",106.0,NJ,07631
29,"Englewood, NJ 07631, United States",40.892877,-73.972638,40.894148,-73.971367,40.891448,-73.974067,https://maps.gstatic.com/mapfiles/place_api/ic...,8cdd64797753c23da90380187c939ae06ec65e7a,Balanced Bodies Fitness Co,...,ChIJodQjFRjxwokRokOnL8wAuWo,NaN,NaN,NaN,5.0,ChIJodQjFRjxwokRokOnL8wAuWo,"['gym', 'health', 'point_of_interest', 'establ...",1.0,NJ,07631
30,"106 Grand Ave, Englewood, NJ 07631, United States",40.889702,-73.976014,40.891114,-73.974601,40.888414,-73.977301,https://maps.gstatic.com/mapfiles/place_api/ic...,13cf211ebb75319085667de670ddadc52b849638,Kaeos Fitness,...,ChIJcR2vyyPxwokRiINh6XgRJGw,"V2QF+VH Englewood, New Jersey",87G8V2QF+VH,NaN,5.0,ChIJcR2vyyPxwokRiINh6XgRJGw,"['gym', 'health', 'point_of_interest', 'establ...",24.0,NJ,07631
173,"521 River Rd, Edgewater, NJ 07020, United States",40.811684,-73.981750,40.813034,-73.980400,40.810334,-73.983100,https://maps.gstatic.com/mapfiles/place_api/ic...,8eb20d2f5254ce7df405697c88a1f618cf3d418f,Fitness Factory Health Club,...,ChIJJ116rUr2wokR6vAvy8iI5s0,"R269+M8 Edgewater, New Jersey",87G8R269+M8,NaN,4.2,ChIJJ116rUr2wokR6vAvy8iI5s0,"['gym', 'health', 'point_of_interest', 'establ...",129.0,NJ,07020
174,"725 River Rd, Edgewater, NJ 07020, United States",40.820079,-73.977218,40.820945,-73.976293,40.817480,-73.978992,https://maps.gstatic.com/mapfiles/place_api/ic...,5a22415a47033095b8f1f437726e614138eef8e5,Orangetheory Fitness,...,ChIJTfsPHEz2wokRj9Z4W9j9VsM,"R2CF+24 Edgewater, New Jersey",87G8R2CF+24,NaN,4.7,ChIJTfsPHEz2wokRj9Z4W9j9VsM,"['gym', 'health', 'point_of_interest', 'establ...",214.0,NJ,07020
284,"103 N Summit St, Tenafly, NJ 07670, United States",40.935710,-73.960193,40.937006,-73.958900,40.934307,-73.961599,https://maps.gstatic.com/mapfiles/place_api/ic...,e7948ef095455ee56a0f9cfe34a2caef484f115e,Retro Fitness,...,ChIJRX9ybcLxwokRhWoVtXj4uA0,"W2PQ+7W Tenafly, New Jersey",87G8W2PQ+7W,NaN,4.2,ChIJRX9ybcLxwokRhWoVtXj4uA0,"['gym', 'health', 'point_of_interest', 'establ...",111.0,NJ,07670
1026,"Zero, Bond St, New York, 10012, United States",40.727083,-73.994790,40.728395,-73.993473,40.725695,-73.996173,https://maps.gstatic.com/mapfiles/place_api/ic...,afd8c88eb9f463d3a620c29fe6e176e4b28277cc,Equinox Bond Street,...,ChIJtxFuAJBZwokRwaNFoBv6Jtc,P2G4+R3 New York,87G8P2G4+R3,NaN,3.9,ChIJtxFuAJBZwokRwaNFoBv6Jtc,"['gym', 'spa', 'health', 'point_of_interest', ...",53.0,NaN,NaN
1130,"316 Sinatra Dr, Hoboken, NJ 07030, United States",40.739962,-74.027059,40.741298,-74.025635,40.738599,-74.028334,https://maps.gstatic.com/mapfiles/place_api/ic...,0e3462615e3d076b2c81c0fee2ff7df59c4ab762,UFC GYM Hoboken,...,ChIJdWsuVeBZwokRh8sJINJD4Ck,"PXQF+X5 Hoboken, New Jersey",87G7PXQF+X5,NaN,4.2,ChIJdWsuVeBZwokRh8sJINJD4Ck,"['gym', 'health', 'point_of_interest', 'establ...",22.0,NJ,07030
1132,"110 Sinatra Dr, Hoboken, NJ 07030, United States",40

In [128]:
df_all = df_all[df_all['state'] == 'NY']
df_all[df_all['state'] != 'NY']

,formatted_address,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,icon,id,name,...,place_id,plus_code.compound_code,plus_code.global_code,price_level,rating,reference,types,user_ratings_total,state,zipcode


Remove zipcodes that are not in NYC 

In [129]:
all_zips = []
for k, v in nyc_zipcodes.items():
    all_zips += v

In [130]:
len(all_zips) == len(set(all_zips))

True

Gyms that are not in nyc zipcodes

In [139]:
df_all[~df_all['zipcode'].isin(all_zips)][['formatted_address', 'state', 'zipcode']]

formatted_address                  0.0
geometry.location.lat              0.0
geometry.location.lng              0.0
geometry.viewport.northeast.lat    0.0
geometry.viewport.northeast.lng    0.0
geometry.viewport.southwest.lat    0.0
geometry.viewport.southwest.lng    0.0
icon                               0.0
id                                 0.0
name                               0.0
opening_hours.open_now             0.0
photos                             0.0
place_id                           0.0
plus_code.compound_code            0.0
plus_code.global_code              0.0
price_level                        0.0
rating                             0.0
reference                          0.0
types                              0.0
user_ratings_total                 0.0
state                              0.0
zipcode                            0.0
dtype: float64

We will drop these

In [136]:
df_all = df_all[df_all['zipcode'].isin(all_zips) & df_all['zipcode'].notnull()]
df_all

,formatted_address,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,icon,id,name,...,place_id,plus_code.compound_code,plus_code.global_code,price_level,rating,reference,types,user_ratings_total,state,zipcode
0,"298 W 231st St, The Bronx, NY 10463, United St...",40.880539,-73.908334,40.881771,-73.907093,40.879072,-73.909793,https://maps.gstatic.com/mapfiles/place_api/ic...,a3485e7994aa8626891015b990080e6dc9d7b6b3,24 Hour Fitness,...,ChIJEfg3t5XzwokRtR2xHxFYi0Q,"V3JR+6M The Bronx, New York",87G8V3JR+6M,NaN,3.4,ChIJEfg3t5XzwokRtR2xHxFYi0Q,"['gym', 'health', 'point_of_interest', 'establ...",187.0,NY,10463
1,"5520 Broadway, The Bronx, NY 10463, United States",40.877351,-73.905654,40.878701,-73.904305,40.876001,-73.907004,https://maps.gstatic.com/mapfiles/place_api/ic...,81389fd5d6b5dd805551b9beb642637143f06a5c,Blink Fitness,...,ChIJKftkWJHzwokRWTI8LcCn9LI,"V3GV+WP The Bronx, New York",87G8V3GV+WP,NaN,3.8,ChIJKftkWJHzwokRWTI8LcCn9LI,"['gym', 'health', 'point_of_interest', 'establ...",211.0,NY,10463
2,"5500 Broadway, The Bronx, NY 10463, United States",40.876568,-73.906144,40.877848,-73.904852,40.875149,-73.907552,https://maps.gstatic.com/mapfiles/place_api/ic...,93a5f9fa33efd28fa5726eedeaae0a52803eb091,Astral Fitness,...,ChIJqczPWpHzwokR3qB2P5FDkIU,"V3GV+JG The Bronx, New York",87G8V3GV+JG,NaN,4.2,ChIJqczPWpHzwokR3qB2P5FDkIU,"['gym', 'health', 'point_of_interest', 'establ...",31.0,NY,10463
3,"82 W 225th St, The Bronx, NY 10463, United States",40.874076,-73.909262,40.875480,-73.907841,40.872780,-73.910541,https://maps.gstatic.com/mapfiles/place_api/ic...,912d8b13c868012e545577eeaa8be0762506adfb,Planet Fitness,...,ChIJr3JBXpLzwokR_dIaA6b0vd8,"V3FR+J7 The Bronx, New York",87G8V3FR+J7,NaN,4.2,ChIJr3JBXpLzwokR_dIaA6b0vd8,"['gym', 'spa', 'health', 'point_of_interest', ...",309.0,NY,10463
4,"453 W 238th St, The Bronx, NY 10463, United St...",40.887410,-73.905057,40.888704,-73.903729,40.886005,-73.906429,https://maps.gstatic.com/mapfiles/place_api/ic...,a272c6a36fa158e12f45a31e7a2bc01896e26c8c,CrossFit La Caja,...,ChIJIUCs9LvzwokRLW1jjLy1ILQ,"V3PV+XX The Bronx, New York",87G8V3PV+XX,NaN,4.9,ChIJIUCs9LvzwokRLW1jjLy1ILQ,"['gym', 'health', 'point_of_interest', 'establ...",14.0,NY,10463
5,"451 W 238th St, The Bronx, NY 10463, United St...",40.887441,-73.905033,40.888719,-73.903712,40.886019,-73.906412,https://maps.gstatic.com/mapfiles/place_api/ic...,bd460b8485fe2617f94ef94eb41db87489a7ae9f,Bronx Barbell Club,...,ChIJCxuojLvzwokRuDZAKdRM9kM,"V3PV+XX The Bronx, New York",87G8V3PV+XX,NaN,4.6,ChIJCxuojLvzwokRuDZAKdRM9kM,"['gym', 'health', 'point_of_interest', 'establ...",9.0,NY,10463
6,"3333 Henry Hudson Parkway East # 2, The Bronx,...",40.885582,-73.914337,40.886918,-73.912778,40.884218,-73.915478,https://maps.gstatic.com/mapfiles/place_api/ic...,1ece03b83790fbe1f8456195d9c8b6b9ac32d7d4,Whitehall Club,...,ChIJ5eQBQ8DzwokR3TulqszkeVs,V3PP+67 New York,87G8V3PP+67,NaN,4.6,ChIJ5eQBQ8DzwokR3TulqszkeVs,"['gym', 'health', 'point_of_interest', 'establ...",19.0,NY,10463
7,"2600 Netherland Ave, The Bronx, NY 10463, Unit...",40.878628,-73.914568,40.880124,-73.913392,40.877424,-73.916092,https://maps.gstatic.com/mapfiles/place_api/ic...,8a85344ff5f2f8d95a5b7c97fe83bbe2991e5ad7,TCR The Club of Riverdale,...,ChIJD-_1fOvzwokRXbzmmmtKaLo,"V3HP+F5 The Bronx, New York",87G8V3HP+F5,NaN,4.0,ChIJD-_1fOvzwokRXbzmmmtKaLo,"['spa', 'restaurant', 'food', 'school', 'gym',...",32.0,NY,10463
8,"5625-2 Broadway, The Bronx, NY 10463, United S...",40.879859,-73.904360,40.881174,-73.902925,40.878474,-73.905624,https://maps.gstatic.com/mapfiles/place_api/ic...,80b6f8991783ed2a98b6bc88be11e91d504ec1eb,Build N Box,...,ChIJgWN3WpbzwokRO9GDlGXTz1M,"V3HW+W7 The Bronx, New York",87G8V3HW+W7,NaN,5.0,ChIJgWN3WpbzwokRO9GDlGXTz1M,"['gym', 'health', 'point_of_interest', 'establ...",97.0,NY,10463
9,"3501 Riverdale Ave, The Bronx, NY 10463, Unite...",40.885412,-73.907463,

In [137]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1814 entries, 0 to 2663
Data columns (total 22 columns):
formatted_address                  1814 non-null object
geometry.location.lat              1814 non-null float64
geometry.location.lng              1814 non-null float64
geometry.viewport.northeast.lat    1814 non-null float64
geometry.viewport.northeast.lng    1814 non-null float64
geometry.viewport.southwest.lat    1814 non-null float64
geometry.viewport.southwest.lng    1814 non-null float64
icon                               1814 non-null object
id                                 1814 non-null object
name                               1814 non-null object
opening_hours.open_now             1442 non-null object
photos                             1447 non-null object
place_id                           1814 non-null object
plus_code.compound_code            1810 non-null object
plus_code.global_code              1810 non-null object
price_level                        30 non-null 

Add borough to gym

In [153]:
zipcode_to_boro = {}

for boro, zipcodes in nyc_zipcodes.items():
    for zipcode in zipcodes:
        zipcode_to_boro[zipcode] = boro
        
zipcode_to_boro

{'10001': 'Manhattan',
 '10002': 'Manhattan',
 '10003': 'Manhattan',
 '10004': 'Manhattan',
 '10005': 'Manhattan',
 '10006': 'Manhattan',
 '10007': 'Manhattan',
 '10009': 'Manhattan',
 '10010': 'Manhattan',
 '10011': 'Manhattan',
 '10012': 'Manhattan',
 '10013': 'Manhattan',
 '10014': 'Manhattan',
 '10016': 'Manhattan',
 '10017': 'Manhattan',
 '10018': 'Manhattan',
 '10019': 'Manhattan',
 '10020': 'Manhattan',
 '10021': 'Manhattan',
 '10022': 'Manhattan',
 '10023': 'Manhattan',
 '10024': 'Manhattan',
 '10025': 'Manhattan',
 '10026': 'Manhattan',
 '10027': 'Manhattan',
 '10028': 'Manhattan',
 '10029': 'Manhattan',
 '10030': 'Manhattan',
 '10031': 'Manhattan',
 '10032': 'Manhattan',
 '10033': 'Manhattan',
 '10034': 'Manhattan',
 '10035': 'Manhattan',
 '10036': 'Manhattan',
 '10037': 'Manhattan',
 '10038': 'Manhattan',
 '10039': 'Manhattan',
 '10040': 'Manhattan',
 '10044': 'Manhattan',
 '10065': 'Manhattan',
 '10075': 'Manhattan',
 '10128': 'Manhattan',
 '10280': 'Manhattan',
 '10301': '

In [155]:
df_all['borough'] = df_all['zipcode'].map(zipcode_to_boro)
df_all[['borough','zipcode']]

,borough,zipcode
0,Bronx,10463
1,Bronx,10463
2,Bronx,10463
3,Bronx,10463
4,Bronx,10463
5,Bronx,10463
6,Bronx,10463
7,Bronx,10463
8,Bronx,10463
9,Bronx,10463


Export as csv

In [156]:
#export to excel and csv
df_all.to_csv("gyms/all_nyc_gyms.csv", index=False)

#### Example data available from a result

In [63]:
pages[0]['results'][0].keys()

dict_keys(['formatted_address', 'geometry', 'icon', 'id', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'rating', 'reference', 'types', 'user_ratings_total'])

In [64]:
pprint.pprint(pages[0]['results'][0])

{'formatted_address': '2402 86th St, Brooklyn, NY 11214, United States',
 'geometry': {'location': {'lat': 40.5988114, 'lng': -73.9889699},
              'viewport': {'northeast': {'lat': 40.60021577989271,
                                         'lng': -73.98756167010728},
                           'southwest': {'lat': 40.59751612010727,
                                         'lng': -73.99026132989272}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
 'id': '85b039b611fbb0b4a82066e0f904535de6882f44',
 'name': 'Dolphin Fitness Clubs',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 3024,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/104557997108262991258/photos">Luke '
                                   'Constantino</a>'],
             'photo_reference': 'CmRaAAAASKrTfz0QaBVgyOJfuknYqNpvS-_XG1SyS-pxfc31F5OQMCbOZ01IWZYkhyiT4kwk0h3kpYZzu87txAxFzUvnAi_SY6tGk-u

In [10]:
pprint.pprint(places_result['results'])

[{'formatted_address': '2402 86th St, Brooklyn, NY 11214, United States',
  'geometry': {'location': {'lat': 40.5988114, 'lng': -73.9889699},
               'viewport': {'northeast': {'lat': 40.60021577989271,
                                          'lng': -73.98756167010728},
                            'southwest': {'lat': 40.59751612010727,
                                          'lng': -73.99026132989272}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
  'id': '85b039b611fbb0b4a82066e0f904535de6882f44',
  'name': 'Dolphin Fitness Clubs',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 3024,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/104557997108262991258/photos">Luke '
                                    'Constantino</a>'],
              'photo_reference': 'CmRaAAAAsmL0MKDv9W9XbBqYT4IyZ5DF4V_GjLlHWzBXLJ0zWz1iWesEiEicLHxwpKakDPweH_yy0-1CxsCaJ8Gl

In [16]:
places_result['next_page_token']

'CpQCAQEAABFHS41uKFe-3bne9XFSQ88VfCJgRGu3JB6ReFquf8y85IlAWsqAYOqXiA30eG2Ceh9LLVB71qcdrTETH3wRMhuLow42DKYidJvtPatC-NxZlBYgPE059pZEtDrgqzUfkuxeiM2RtICpFYlk8Kir86kGc1lvj7Jz6Z9cNr4SGkeigwLJPksCJlunyQUR6bVlbhYWi4kF097RXzfKZSe_KkXhLjJ7UWAS6k0ZgAvK1Ao8DYTiH7Qou0MMjP8k4KlNWTrTxAdSJ5--ZavozVKV_4sNVg2VyZ8wR3PhSbx99KWmK4oMi1tCBCEGdvEY7Ko19O3o1PpyuLvhqfs02429Sn_oiP7wgRDCf0WM6r1FQTxSEhCaUKaN33QqOuPjiOlKj0UOGhQV8bcpMWnPZJkrHsFB2BU4JCCsfA'